In [7]:
from datetime import datetime, timedelta
from pathlib import Path

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.forcing.winds import BestTrackForcing
from adcircpy.utilities import download_mesh

import numpy as np

stage = 'basemodel'
HURRICANE = 'Matthew2016'
OUTPUT_DIRECTORY = stage+'/runs/'+HURRICANE

In [8]:
# open mesh file
mesh = AdcircMesh.open('datasets/nc_inundation_v6c.grd', crs=4326)    

# Adding Manning's
# generate tau0 factor
mesh.generate_tau0()

# also add Manning's N to the domain (constant for this example)
mesh.mannings_n_at_sea_floor = np.full(mesh.values.shape, 0.025)

# initialize tidal forcing and constituents
tidal_forcing = Tides()
tidal_forcing.use_all()
mesh.add_forcing(tidal_forcing)

/global/home/users/alexandregeorges/.local/lib/python3.11/site-packages/adcircpy/mesh/parsers/grd.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['nodes'] = pandas.read_csv(
/global/home/users/alexandregeorges/.local/lib/python3.11/site-packages/adcircpy/mesh/parsers/grd.py:37: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['elements'] = pandas.read_csv(


In [9]:
# set simulation dates
spinup_time = timedelta(days=15)                                                                        
duration = timedelta(days=10)
start_date = datetime(2016, 9, 28, 18)
end_date = start_date + duration

In [10]:
# instantiate driver object
driver = AdcircRun(mesh, start_date, end_date, spinup_time)                                                                                                                                                     # write driver state to disk
driver.write(OUTPUT_DIRECTORY, overwrite=True)

In [11]:
bash_script = rf"""
#!/bin/bash

# Rename and modify fort.15 for netcdf output and wind run
mv fort.15.coldstart fort.15

replacement_ele="-3 0.000000 15.000000 20                              ! NOUTGE"
replacement_vel="-3 0.000000 15.000000 20                              ! NOUTGV"

sed -i "s/0 0.000000 0.000000 0                                           ! NOUTGE/$replacement_ele/" fort.15
sed -i "s/0 0.000000 0.000000 0                                           ! NOUTGV/$replacement_vel/" fort.15


current_dir=$(pwd)
folder_name=$(basename $current_dir)

mkdir ../../logs
mkdir ../../logs/$folder_name
log_header="Logs of run attempts and their parameters."
log_header2="Hurricane {HURRICANE}"
log_header3="------------------------------------------"
log_header4="id\t\t\t\t\t\tdt\t\t\t\t\t\ttype\t\t\t\t\t\tcomment"
echo -e $log_header > runlog.out
echo -e $log_header2 >> runlog.out
echo -e $log_header3 >> runlog.out
echo -e $log_header4 >> runlog.out

cp ../../../scripts/run.sh run.sh
cp ../../../scripts/slurmcold.sh slurmcold.sh
cp ../../../scripts/slurmhot.sh slurmhot.sh

current_dir=$(pwd)
folder_name=$(basename $current_dir)
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name base/" slurmcold.sh
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name base hot/" slurmhot.sh

chmod +x run.sh slurmcold.sh slurmhot.sh
"""

# Write the bash script to the output directory
with open(OUTPUT_DIRECTORY + '/init.sh', 'w') as file:
    file.write(bash_script)

print("Bash script created and outputted at", OUTPUT_DIRECTORY)

Bash script created and outputted at basemodel/runs/Matthew2016
